In [21]:
# https://www.kaggle.com/gulsahdemiryurek/harry-potter-dataset

In [22]:
from pathlib import Path
from collections import Counter
import numpy as np
import nltk
import os
import csv

nltk.download('vader_lexicon')

root_dir = Path(os.path.abspath('__file__')).parent
dataset_dir = root_dir / 'data' / 'book'
book1_dir = dataset_dir / 'Harry Potter 1.csv'
book2_dir = dataset_dir / 'Harry Potter 2.csv'
book3_dir = dataset_dir / 'Harry Potter 3.csv'

print(root_dir)
print(dataset_dir)

/home/shane/workspace/sna/potter
/home/shane/workspace/sna/potter/data/book


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/shane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

with open(book3_dir, 'r') as f:
    book = list(f.readlines())[1:]
    book = list(map(lambda x: x.split(';'), book))
    book = [list(map(lambda x: x.strip(), x)) for x in book]

characters = []

edges = []
prev_character = ""
score_sum = 0
for character, sentence in book:
    characters.append(character.title())
    if prev_character != character:
        if prev_character:
            edges.append([f"{prev_character.title()}__{character.title()}", score_sum])
        prev_character = character
        score_sum = 0
    score_sum += 1 + abs(analyser.polarity_scores(sentence)['compound'])

edges_dict = {}
for k, v in edges:
    if k in edges_dict:
        edges_dict[k] += v
    else:
        edges_dict[k] = v

character_dict = dict(zip(dict(sorted(dict(Counter(characters)).items(), key=lambda x: x[1], reverse=True)).keys(), list(range(1, len(characters)+1))))

In [24]:
import math

# *Vertices 4
# 1 "George"
# 2 "Susan"
# 3 "John"
# 4 "Sarah"
# *Edges
# 1 2 1
# 2 3 1
# 3 4 1

with open(root_dir/'book3_logscale.net', 'w') as f:
    f.write(f"*Vertices\t{len(character_dict)}\n")
    for name, i in list(character_dict.items()):
        f.write(f"{i}\t\"{name}\"\n")
    f.write(f"*Edges\n")
    for from_to, score in list(sorted(edges_dict.items(), key=lambda x: x[1], reverse=True)):
        from_character, to_character =  from_to.split('__')
        f.write(f"{character_dict[from_character]}\t{character_dict[to_character]}\t{math.log2(score + 1)}\n")